In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#supress warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df_full = pd.read_csv('/kaggle/input/google-playstore-apps/Google-Playstore-Full.csv')
df_full.head()

In [ ]:
df_full.columns

Seems excel has created 4 empty columns,lets confirm if they are really empty

# Analysing 4 undefined columns

In [ ]:
print(df_full.shape)
df_full.isna().sum()

There are few values in those 4 columns, lets view them

In [ ]:
# column Unnamed: 11 

un11 = df_full['Unnamed: 11'][df_full['Unnamed: 11'].notna()]
print(f'No.of records in Unnamed: 11 ->  {len(un11)} \n')
print(f'% of data present -> {(len(un11) / len(df_full) * 100)}\n')
print(un11.head(3))
print('\n')
df_full[df_full['Unnamed: 11'].notna()].head(3)

The values are shifted to another columns, for example
Index no:6941,<br> 
Rating            ->   Size <br>
Installs          ->   Content Rating <br>
Size              ->   Last Updated <br>
Last Updated      ->   Unnamed: 11 <br>
Minimum Version   ->   Unnamed: 12 <br>
Latest Version    ->   Unnamed: 13 etc.



In [ ]:
# column Unnamed: 12

un12 = df_full['Unnamed: 12'][df_full['Unnamed: 12'].notna()]
print(f'No.of records in Unnamed: 12 ->  {len(un12)} \n')
print(f'% of data present -> {(len(un12) / len(df_full) * 100)}\n')
print(un12)

In [ ]:
# column Unnamed: 13

un13 = df_full['Unnamed: 13'][df_full['Unnamed: 13'].notna()]
print(f'No.of records in Unnamed: 13 ->  {len(un13)} \n')
print(f'% of data present -> {(len(un13) / len(df_full) * 100)}\n')
print(un13)

In [ ]:
# column Unnamed: 14

un14 = df_full['Unnamed: 14'][df_full['Unnamed: 14'].notna()]
print(f'No.of records in Unnamed: 14 ->  {len(un14)} \n')
print(f'% of data present -> {(len(un14) / len(df_full) * 100)}\n')
print(un14)

From the above observatios, it's very clear the values in the undefined columns are swifted from the original columns.<br>
Since the values are very very few (max % is 0.006), we can either delete entire rows or can place the values where they supposed to be.
In this case, placing them properly is little tricky, i choose to delete them since deleting them does not the model performance.

In [ ]:
un_11 = df_full[df_full['Unnamed: 11'].notna()].index
un_12 = df_full[df_full['Unnamed: 12'].notna()].index
un_13 = df_full[df_full['Unnamed: 13'].notna()].index
un_14 = df_full[df_full['Unnamed: 14'].notna()].index

#pass them into set to remove duplicates
un_index = set(list(un_11) + list(un_12) + list(un_13) + list(un_14))

In [ ]:
# delete undefined columns and NA's
print(f'Before Delete : {df_full.shape}')
df_full.drop(un_index,inplace=True)
df_full.drop(columns=['Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14'],axis=1,inplace=True)
print(f'After Delete : {df_full.shape}')

In [ ]:
df_full.head()

In [ ]:
df_full.info()

First glance at head() tells us some of the fetures are numerical (Rating,Reviews,Installs and price) but may be presence of some text in them turned to object type,which was shown in info().

In [ ]:
df_full.describe()

App Name : Seems of missing(????) values, and duplicate records ( > 20k (count - unique)) <br>
Category : most of the apps are EDUCATION realted.<br>
Rating   : Highest rating given to apps is 5 <br>
Reviews  : top review given to apps is 1, this might be either the apps are new or failed to reach/satify users<br>
Size : some apps,the size varies with device.<br>
Price : ~96% of the apps are free to use <br>
Content Rating : 90% of the apps are made to use everyone.



In [ ]:
df_full.isna().sum()

In [ ]:
# df_full.columns = df_full.columns.str.replace(' ', '_')

## **App Name**

In [ ]:
df_full[df_full['App Name'].isna()]

Only one records has NA.

In [ ]:
df_full.dropna(axis=0,subset=['App Name'],inplace=True)

In [ ]:
df_full['App Name'].value_counts().head(25)

In [ ]:
df_full[df_full['App Name'].isin(['????'])].head()

In [ ]:
wrong_df = df_full[df_full['App Name'].str.contains('?',regex = False)]
wrong_df

In [ ]:
print(f'Total {len(wrong_df)} records i.e. ({round((len(wrong_df)/len(df_full))*100,2)}%) contains "?" in them')
# wrong_df[wrong_df.iloc[:,1:].duplicated(keep=False )]

Some apps does not have a name (i.e ?'s in App Name) , I am not gonna remove those records since only app name is missing but rest of information is still present and it's  valid informationand, this coulmn is not a good predictor.We can drop it.

In [ ]:
# dropping App Name columns
# df_full.drop(columns=['App Name'],inplace=True)

In [ ]:
# ques_index = df_full[df_full['App Name'].str.contains('?',regex = False)].index
# df_full.drop(index=ques_index,inplace=True)

In [ ]:
# df_full.head(15)

In [ ]:
# checking for duplicates
df_full[df_full.duplicated(subset='App Name')].sort_values(by=['App Name'])

There are 22643 (8.5%) Apps seems duplicated, let's confirm if all of them are really duplicated.

In [ ]:
# Lets check with some apps instead of all at once
df_full[df_full['App Name'].isin(['#NAME?'])].sort_values(by='App Name')

In [ ]:
df_full[df_full['App Name'].isin(['??'])].sort_values(by='App Name').head()

It's very clear now that duplicated count we saw was false, most of these records does not have app names, deleting them blindly cause lose of information. 

In [ ]:
# df_full[df_full.duplicated()]
# checking for duplicates
dupli = df_full[df_full.duplicated()].sort_values(by=['Category','Rating','Last Updated'])
print(len(dupli))
dupli

There are total 15 duplicated records, We should be very careful with them coz they may lead to **Data Leakage**.

In [ ]:
#drop duplicates
df_full.drop_duplicates(inplace=True)

In [ ]:
# checking for duplicates
df_full[df_full.duplicated()].sort_values(by=['Category','Rating','Last Updated'])

In [ ]:
df_full.shape

## **Category**

In [ ]:
df_full.Category.unique()

No suspicious values in Category columns, everythong looks fine. 

In [ ]:
df_full.Category.value_counts()

In [ ]:
df_full.Category.value_counts(normalize=True)[:10]

In [ ]:
plt.figure(figsize=(12,5))
p = sns.set(style="darkgrid")
p = sns.countplot(x='Category',data=df_full)
_ = plt.setp(p.get_xticklabels(), rotation=90)  # Rotate labels
plt.title('App Category',size = 20);

1. Almost 13% of the apps in play store are related to Education.
2. Top 3 Categories in the play store are EDUCATION, TOOLS and BOOKS_AND_REFERENCE.

## **Rating**

In [ ]:
df_full.Rating.unique()

In [ ]:
df_full.isna().sum()

No NA's in Rating Column.

In [ ]:
df_full.Rating.value_counts()

Top rating given in the Play-store is 5.

In [ ]:
df_full.info()

In [ ]:
# Since its a numerical columns, lets convert it to float
df_full.Rating = pd.to_numeric(df_full.Rating,errors='coerce')

In [ ]:
df_full.Rating.isna().sum()

In [ ]:
df_full.Rating.describe()

The rating is on a scale on 1-5, with 1 being minimum and 5 being maximum. The mean rating is 4.27 and the median rating is 4.4, i.e average rating is above 4.4

In [ ]:
df_full.sort_values(by=['Rating'],ascending=False).head()

Some of the Apps are rated 5,they are not popular as they have very less reviews.

In [ ]:
# Review distibution 
g = sns.kdeplot(df_full.Rating,shade=True,color='blue')
g.set_xlabel("Rating")
g.set_ylabel("Frequency")
plt.title('Distribution of Rating',size = 20);

It tells same thing what i commented above.

## **Reviews**

In [ ]:
df_full.info()

1. No NA's present. 
2. Reviews supposed to be an integer, but its an object, lets understand it.

In [ ]:
# df_full[(df_full.Reviews.astype('str').str.isnumeric())]

# Since its a numerical columns, lets convert it to float
df_full.Reviews = pd.to_numeric(df_full.Reviews,errors='coerce',downcast='integer')

In [ ]:
df_full.Reviews.head()

In [ ]:
df_full.Reviews.isna().sum()

In [ ]:
df_full.Reviews.describe()

In [ ]:
df_full.sort_values(by='Reviews',ascending=False).head()

We see Whatsapp, Facebook, Instagram, messenger and Clash of clans have highest reviews. All these apps are freely available and their category is  social, communication and Game.

In [ ]:
# Review distibution 
g = sns.kdeplot(df_full.Reviews,shade=True,color='blue')
g.set_xlabel("Reviews")
g.set_ylabel("Frequency")
plt.title('Distribution of Review',size = 20);

## **Installs**

In [ ]:
# df_full.info()

In [ ]:
df_full.Installs.value_counts(dropna=False)

There are some special characters to remove like  **,**  and  **+**

In [ ]:
# remove "," and "+"
df_full.Installs = df_full.Installs.str.replace(',','').str.replace('+','')

In [ ]:
df_full.Installs.value_counts(dropna=False)

In [ ]:
# convert to numerical 
df_full.Installs = pd.to_numeric(df_full.Installs,errors='coerce')

In [ ]:
# TODO

# pandas replace can replace list - list
# df_full['Install_cat'] = df_full.Installs.replace(sorted(df_full.Installs.unique()),range(0,len(sorted(df_full.Installs.unique())),1))

In [ ]:
df_full.info()

In [ ]:
df_full.sort_values(by='Installs',ascending=False).head()

All TOP 5 installed Apps are Google Products like Google Search, Google Maps, You Tube, Google Play Movies & TV and Googel Photos.

In [ ]:
# Review distibution 
g = sns.kdeplot(df_full.Installs,shade=True,color='blue')
g.set_xlabel("Installs")
g.set_ylabel("Frequency")
plt.title('Distribution of Installs',size = 20);

Most of the apps (93%) have below 10lac installations

## **Size**

In [ ]:
pd.set_option('display.max_rows', 2000)

In [ ]:
df_full.Size.value_counts()[:15]

In [ ]:
df_full.Size[~(df_full.Size.str.contains('M') | df_full.Size.str.contains('k'))]

In [ ]:
# (11726 / len(df_full))*100

In [ ]:
df_full.Size[(df_full.Size.str.contains(','))].head()

1. Size of the apps are in either KB's or MB's. Lets convert all apps in KB's (Kilo Bytes).
2. Almost 4% of the apps size is depends on the handset.
3. "," is present in for values.

In [ ]:
# df_full.Size.unique()

In [ ]:
kb_index = df_full.Size[df_full.Size.str.contains('k')].index
mb_index = df_full.Size[(df_full.Size.str.contains('M'))].index
print(f"No.of App's in KB's : {len(kb_index)} ({round((len(kb_index)/len(df_full))*100,2)}%)")
print(f"No.of App's in MB's : {len(mb_index)} ({round((len(mb_index)/len(df_full))*100,2)}%)")

In [ ]:
df_full.Size = df_full.Size.str.replace('M','')
df_full.Size = df_full.Size.str.replace('k','')
df_full.Size = df_full.Size.str.replace(',','')

In [ ]:
df_full.Size.value_counts()[:5]
# tail charachter has removed

In [ ]:
df_full.Size[(df_full.Size.str.contains(','))].head()
# comma has removed

In [ ]:
# to_numeric() converts to float(default) and non-numeric values will be replaced with NAN 
df_full.Size = pd.to_numeric(df_full.Size,errors='coerce')

In [ ]:
df_full.Size.head()

In [ ]:
df_full.Size.value_counts(dropna=False).head()
# df_full.Size.isna().sum()

'Varies with device' is replaced with NA's. We can either delete them or impute them with some advanced techniques like KNNImputer or impyute. In this case, I'll check the accuracy with and without remove them.

In [ ]:
#holding NA's in a separate DataFrame
df_full_NA = df_full.copy()

In [ ]:
# converting the size MB to KB (1 MB = 1000 KB)
df_full.Size.loc[mb_index] = df_full.Size.loc[mb_index] * 1000
df_full.Size.head()

In [ ]:
# Drop NA's w.r.t Size column
df_full.dropna(subset=['Size'],inplace=True)

In [ ]:
df_full.Size.describe()

1. MIN size of an App is 3KB and MAX is 347MB, AVG size of an app is ~16MB.

In [ ]:
df_full.Size.head()

In [ ]:
df_full.Size.isna().sum()

All NA's in Size column dropped.

In [ ]:
# review the shape after removing of NA's
df_full.shape

In [ ]:
df_full.sort_values(by='Size',ascending=False).head(2)

The Size of ***Jurassic World Alive*** and ***Agent A: A puzzle in disguise*** apps is over 330MB , and both are GAMING apps.(seems gaming apps are bigger in size)

In [ ]:
# Size distibution 
g = sns.kdeplot(df_full.Size,shade=True,color='blue')
g.set_xlabel("Size")
g.set_ylabel("Frequency")
plt.title('Distribution of Size',size = 20);

As i already mentioned the 2 gaming apps have over 330MB of size, which we can see grapically the long right tail at around 340MB. 

## **Price**

In [ ]:
df_full.Price.value_counts().head()

$ is a special charachters in Price column that we should remove.

In [ ]:
df_full.Price = df_full.Price.str.replace('$','')

In [ ]:
df_full.Price = pd.to_numeric(df_full.Price,errors='coerce')

In [ ]:
df_full.Price.value_counts().head()

In [ ]:
df_full.info()

In [ ]:
df_full.Price.describe()

Seems most of the apps are free to use, and max price of an app is $400

In [ ]:
# df_full.sort_values(by='Price',ascending=False).head(5)
df_full[df_full.Price == 399.99]

Above 5 apps are high priced in the Play-Store. These Apps are less popular as they have low reviews and installs.

In [ ]:
# df_full[df_full.Price.astype('str').str.contains('scott')]

In [ ]:
# Price distibution 
g = sns.kdeplot(df_full.Price,shade=True,color='blue', bw=1.5)
g.set_xlabel("Price")
g.set_ylabel("Frequency")
plt.title('Distribution of Price',size = 20);

## **Content Rating**

In [ ]:
df_full['Content Rating'].value_counts()

In [ ]:
# 231211/len(df_full)

Most of the apps (90%) are made for everyone to use.

In [ ]:
# removing numericals and + operator
df_full['Content Rating'] = df_full['Content Rating'].str.split(n=1,expand=True)[0]

In [ ]:
df_full['Content Rating'].unique()

In [ ]:
df_full[df_full['Content Rating'] == 'Unrated'].head()

In [ ]:
df_full[df_full['Content Rating'] == 'Adults'].head()

In [ ]:
df_full.info()

## **Data Preparation**

In [ ]:
df_full.drop(columns=['App Name','Last Updated','Minimum Version','Latest Version'],inplace=True)

In [ ]:
df_full.head()

In [ ]:
df_full[df_full.duplicated()]

In [ ]:
df_full.drop_duplicates(inplace=True)

In [ ]:
# df_full.Category.value_counts()

## **Encoding Categoricals**
1. One-Hot
2. CatBoostEncoder

In [ ]:
# method : 1 (One-Hot)
df_full_dummy = pd.get_dummies(df_full,columns=['Category','Content Rating'],drop_first=True)
# df_full_dummy = pd.get_dummies(df_full,columns=['Category'],drop_first=True)

In [ ]:
df_full_dummy.head()

In [ ]:
X_dummy = df_full_dummy.drop(columns=['Rating'],axis=1)
y_dummy = df_full_dummy.loc[:,'Rating']
X_dummy.shape, y_dummy.shape

In [ ]:
X_dummy.head(2)

In [ ]:
y_dummy.head(2)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_dummy,y_dummy,test_size=0.3,random_state = 14)

In [ ]:
x_test.head()

In [ ]:
# df_full.head()

In [ ]:
# method : 2 (CatBoostEncoder)
import category_encoders as ce

df_full_cat = df_full.copy()

cbe = ce.CatBoostEncoder(cols=['Category','Content Rating'])
df_full_cat.loc[:,['Category','Content Rating']] = cbe.fit_transform(df_full_cat.loc[:,['Category','Content Rating']],df_full_cat['Rating'])

# cbe = ce.CatBoostEncoder(cols=['Category'])
# df_full_cat.loc[:,['Category']] = cbe.fit_transform(df_full_cat.loc[:,['Category']],df_full_cat['Rating'])

In [ ]:
df_full_cat.head()

In [ ]:
X_cat = df_full_cat.drop(columns = ['Rating'],axis=1)
y_cat = df_full_cat.loc[:,'Rating']
X_cat.shape , y_cat.shape

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X_cat,y_cat,test_size=0.3,random_state = 14)

In [ ]:
# Model traing and predicting
def dummy_model_building(model):
    from sklearn.metrics import mean_squared_error
    model.fit(x_train,y_train)
    print('trained')
    train_score = model.score(x_train , y_train)
    test_score = model.score(x_test , y_test)
    predict = model.predict(x_test)

    print('Train Score on Dummy : {}'.format(train_score))
    print('Test Score on Dummy : {}'.format(test_score))
    print(f'MSE : {mean_squared_error(y_test, predict)}')
#     print(classification_report(y_test, predict))

    print('\n \n')

    try:
        features = X_dummy.columns[:10]
        importances = model.feature_importances_[:10]
        indices = np.argsort(importances)

        plt.title('Feature Importances')
        plt.barh(range(len(indices)), importances[indices], color='b', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.xlabel('Relative Importance')
        plt.show()
    except :
        print('This model does not support Feature Selection')

# cat score function
def cat_model_building(model):
    from sklearn.metrics import mean_squared_error
    model.fit(X_train,Y_train)
    train_score_ = model.score(X_train , Y_train)
    test_score_ = model.score(X_test , Y_test)
    predict_ = model.predict(X_test)

    print('Train Score on Cat_encode : {}'.format(train_score_))
    print('Test Score on Cat_encode : {}'.format(test_score_))
#     print(confusion_matrix(Y_test, predict_))
    print(f'MSE : {mean_squared_error(Y_test, predict_)}')
#     print(classification_report(Y_test, predict_))

    print('\n \n')

    try:
        features = X_cat.columns
        importances = model.feature_importances_
        indices = np.argsort(importances)

        plt.title('Feature Importances')
        plt.barh(range(len(indices)), importances[indices], color='b', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.xlabel('Relative Importance')
        plt.show()
    except :
        print('This model does not support Feature Selection')

In [ ]:
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dummy_model_building(dt)
print('\n')
cat_model_building(dt)

In [ ]:
# LinearRegression
from sklearn.linear_model import LinearRegression
le = LinearRegression()
dummy_model_building(le)
print('\n')
cat_model_building(le)

In [ ]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
dummy_model_building(rf)
print('\n')
cat_model_building(rf)

In [ ]:
# Bagging Classifier
from sklearn.ensemble import BaggingRegressor

bc = BaggingRegressor()
dummy_model_building(bc)
print('\n')
cat_model_building(bc)

In [ ]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

gc = GradientBoostingRegressor()
dummy_model_building(gc)
print('\n')
cat_model_building(gc)

In [ ]:
# AdaBoosting
from sklearn.ensemble import AdaBoostRegressor

ac = AdaBoostRegressor()
dummy_model_building(ac)
print('\n')
cat_model_building(ac)

In [ ]:
# Stacking
from sklearn.ensemble import StackingRegressor
estimators = [('decisiontree', dt), ('randomforest', rf), ('bagging', bc), ('gradientboost', gc), ('Ada Boost', ac)]
sc = StackingRegressor(estimators)
dummy_model_building(sc)
print('\n')
cat_model_building(sc)

All the modesls failed to learn the patterns, i mean the data itself seems do not have patterns. This could be coz of weak featues. May be adding more features will help to improve the model accuracy.